## Primer inciso que pide elegir activos

In [27]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import rbf_kernel
import matplotlib.pyplot as plt
import yfinance as yf
from IPython.display import display, Markdown
%matplotlib inline

In [28]:
tickers=['GOOGL','META','AAPL','NFLX','NVDA']

f_inicial='2023-01-01'
f_final='2023-12-30'

closes=yf.download(tickers,f_inicial,f_final)['Adj Close']

[*********************100%***********************]  5 of 5 completed


In [29]:
closes

Ticker,AAPL,GOOGL,META,NFLX,NVDA
Date,,,,,
2023-01-03 00:00:00+00:00,123.768463,88.899872,124.365547,294.950012,14.305580
2023-01-04 00:00:00+00:00,125.045036,87.862442,126.987663,309.410004,14.739294
2023-01-05 00:00:00+00:00,123.718987,85.987076,126.558952,309.700012,14.255614
2023-01-06 00:00:00+00:00,128.271118,87.124260,129.629700,315.549988,14.849224
2023-01-09 00:00:00+00:00,128.795593,87.802582,129.081345,315.170013,15.617718
...,...,...,...,...,...
2023-12-22 00:00:00+00:00,192.656174,141.140518,352.329163,486.760010,48.819527
2023-12-26 00:00:00+00:00,192.108856,141.170441,353.764832,491.190002,49.268425
2023-12-27 00:00:00+00:00,192.208374,140.023270,356.755859,491.790009,49.406395


In [30]:
ret = closes.pct_change().dropna()
mu = ret.mean()
mu # vector mu

Ticker
AAPL     0.001835
GOOGL    0.001990
META     0.004493
NFLX     0.002288
NVDA     0.005442
dtype: float64

In [31]:
sigma = ret.cov()
sigma # matriz de covarianza

Ticker,AAPL,GOOGL,META,NFLX,NVDA
Ticker,,,,,
AAPL,0.000158,0.000127,0.000170,0.000125,0.000171
GOOGL,0.000127,0.000367,0.000292,0.000146,0.000237
META,0.000170,0.000292,0.000630,0.000192,0.000315
NFLX,0.000125,0.000146,0.000192,0.000561,0.000221
NVDA,0.000171,0.000237,0.000315,0.000221,0.000932


In [32]:
sigma_inv = np.linalg.inv(sigma)
pd.DataFrame(sigma_inv) # matriz de covarianza inversa

,0,1,2,3,4
0,11096.793412,-1679.977768,-1437.331941,-1222.965668,-829.962567
1,-1679.977768,4870.326007,-1563.880516,-215.587248,-353.372718
2,-1437.331941,-1563.880516,2888.693761,-151.443079,-277.813424
3,-1222.965668,-215.587248,-151.443079,2244.205987,-201.881635
4,-829.962567,-353.372718,-277.813424,-201.881635,1456.918917


In [33]:
ones = np.ones(len(mu))
ones # vector de unos

array([1., 1., 1., 1., 1.])

In [34]:
w = (sigma_inv.dot(ones))/(ones.T.dot(sigma_inv).dot(ones))
w # vector de pesos para portafolio de mínima varianza

array([ 0.88608075,  0.15810824, -0.08100094,  0.06762772, -0.03081577])

In [35]:
w_df = pd.DataFrame(w, index=closes.columns, columns=['w'])
w_df.T # pesos en DataFrame

Ticker,AAPL,GOOGL,META,NFLX,NVDA
w,0.886081,0.158108,-0.081001,0.067628,-0.030816


In [36]:
rp = w.T.dot(mu)
rp # rendimiento esperado del portafolio de mínima varianza diario

0.0015635380336027878

In [37]:
mu

Ticker
AAPL     0.001835
GOOGL    0.001990
META     0.004493
NFLX     0.002288
NVDA     0.005442
dtype: float64

## Segundo inciso que pide elegir activos

In [38]:
U = np.array([mu, ones]).T
U

array([[0.00183493, 1.        ],
       [0.00198971, 1.        ],
       [0.00449317, 1.        ],
       [0.00228751, 1.        ],
       [0.0054418 , 1.        ]])

In [39]:
M = U.T.dot(sigma_inv).dot(U)
pd.DataFrame(M)

,0,1
0,0.047738,10.457732
1,10.457732,6688.504957


In [40]:
M_inv = np.linalg.inv(M)
pd.DataFrame(M_inv)

,0,1
0,31.860066,-0.049814
1,-0.049814,0.000227


In [41]:
u = np.array([[rp],
              [1]])
u

array([[0.00156354],
       [1.        ]])

In [42]:
w2 = sigma_inv.dot(U).dot(M_inv).dot(u)
w2

array([[ 0.88608075],
       [ 0.15810824],
       [-0.08100094],
       [ 0.06762772],
       [-0.03081577]])

In [43]:
w2_df = pd.DataFrame(w2, index=closes.columns, columns=['w'])
w2_df.T # pesos en DataFrame

Ticker,AAPL,GOOGL,META,NFLX,NVDA
w,0.886081,0.158108,-0.081001,0.067628,-0.030816
